In [3]:
import plotly
import plotly.express as px
import pandas as pd
from plotly.offline import init_notebook_mode
import folium

init_notebook_mode(connected = True)


In [4]:
#read WPI INTL Students file
isdf = pd.read_excel('../data/wpiintlstudentsmap2.xlsx')

#make values strings
isdf['clat'] = isdf['clat'].astype(str)
isdf['clong'] = isdf['clong'].astype(str)
isdf['country1'] = isdf['country1'].astype(str)

isdf = isdf.rename(columns={'class':'classof'})

#assign dataframe values to variables
hlat = isdf['hlat']
hlong = isdf['hlong']
clat = isdf['clat']
clong = isdf['clong']
wpiclass = isdf['classof']
sname = isdf['sname']
home = isdf['home1']
c1 = isdf['country1']

print(isdf)

                      sname                      home1     country1  \
0    Abadjieff, Ivan Varban     T. Pazordjik, Bulgaria     Bulgaria   
1         Aguirre, Frank M.           Cienfuegos, Cuba         Cuba   
2        Ajemian, Shahin A.                    Armenia      Armenia   
3         Alfonso, Fernando                   Colombia     Colombia   
4            Allen, Kenneth      Yarmouth, Nova Scotia       Canada   
..                      ...                        ...          ...   
184      Yrigoyen, Juan I.   Guatemala City, Guatemala    Guatemala   
185  Yuan, Harold Hsiang-Ho   Tang Shan, Chihli, China        China   
186       Yuan, Tsong-Kyien            Hangchow, China        China   
187              Yuan, Tung              Peking, China        China   
188     Zaragoza, Jose Luis        Manila, Philippines  Philippines   

          clat        clong           home2       hlat       hlong  classof  
0    42.733883     25.48583      Pazardzhik  42.200000   24.333000   

In [5]:
#create map to display all student home city/town and/or country, color based on year graduated
fig = px.scatter_geo(
    isdf,
    lat=hlat,
    lon=hlong,
    color = wpiclass,
    hover_name= sname,
    hover_data=[home, wpiclass],
    #projection = 'conic conformal',
    projection = 'winkel tripel',
)

fig.show()

In [6]:
#concatenate country, latitude, and longitude as column to prepare to pivot

isdf['classof'] = isdf['classof'].astype(str)

isdf['classof'] = isdf['classof'].str[0:4]

isdf['cll'] = isdf['country1'] + '|' + isdf['clat'] + '|' + isdf['clong']
isdf['cllclass'] = isdf['country1'] + '|' + isdf['clat'] + '|' + isdf['clong'] + '|' + isdf['classof']

print(isdf)

                      sname                      home1     country1  \
0    Abadjieff, Ivan Varban     T. Pazordjik, Bulgaria     Bulgaria   
1         Aguirre, Frank M.           Cienfuegos, Cuba         Cuba   
2        Ajemian, Shahin A.                    Armenia      Armenia   
3         Alfonso, Fernando                   Colombia     Colombia   
4            Allen, Kenneth      Yarmouth, Nova Scotia       Canada   
..                      ...                        ...          ...   
184      Yrigoyen, Juan I.   Guatemala City, Guatemala    Guatemala   
185  Yuan, Harold Hsiang-Ho   Tang Shan, Chihli, China        China   
186       Yuan, Tsong-Kyien            Hangchow, China        China   
187              Yuan, Tung              Peking, China        China   
188     Zaragoza, Jose Luis        Manila, Philippines  Philippines   

          clat        clong           home2       hlat       hlong classof  \
0    42.733883     25.48583      Pazardzhik  42.200000   24.333000   

In [7]:
#pivot data based on country using student name as index and Class Of year as value
pivot = isdf.pivot(index='sname', columns='cll', values='classof')

#count the number of times the country is listed
countc = pivot.count()
#convert series to dataframe
countdf = countc.to_frame()

#make cll index to column
countdf.reset_index(inplace=True)

#rename column
countdf = countdf.rename(columns={0:'country_count'})
countdf.columns

print(countdf)

                                                cll  country_count
0                   Argentina|-38.416097|-63.616672              1
1                       Armenia|40.069099|45.038189              3
2                   Australia|-25.274398|133.775136              6
3                       Austria|47.516231|14.550072              1
4                        Belgium|50.503887|4.469936              1
5                       Brazil|-14.235004|-51.92528             13
6                       Bulgaria|42.733883|25.48583              1
7                      Canada|56.130366|-106.346771             33
8                       Chile|-35.675147|-71.542969              3
9                         China|35.86166|104.195397             31
10                     Colombia|4.570868|-74.297333              2
11                   Costa Rica|9.748917|-83.753428              1
12                        Cuba|21.521757|-77.781167              6
13               Czech Republic|49.817492|15.472962           

In [8]:
countdf[['c2','latc','lonc']] = countdf.cll.str.split("|",expand=True)
print(countdf)

                                                cll  country_count  \
0                   Argentina|-38.416097|-63.616672              1   
1                       Armenia|40.069099|45.038189              3   
2                   Australia|-25.274398|133.775136              6   
3                       Austria|47.516231|14.550072              1   
4                        Belgium|50.503887|4.469936              1   
5                       Brazil|-14.235004|-51.92528             13   
6                       Bulgaria|42.733883|25.48583              1   
7                      Canada|56.130366|-106.346771             33   
8                       Chile|-35.675147|-71.542969              3   
9                         China|35.86166|104.195397             31   
10                     Colombia|4.570868|-74.297333              2   
11                   Costa Rica|9.748917|-83.753428              1   
12                        Cuba|21.521757|-77.781167              6   
13               Cze

In [9]:
#create bubble map to show different sizes of circles depending on number of students from the country from before WWII
figc = px.scatter_geo(
    countdf,
    lat=countdf['latc'],
    lon=countdf['lonc'],
    color=countdf['c2'],
    size=countdf['country_count'],
    hover_name= countdf['c2'],
    hover_data= [countdf['country_count']],
    #projection = 'conic conformal',
    projection = 'winkel tripel',
)

figc.show()

In [10]:
#pivot data based on country using student name as index and Class Of year as value
pivot2 = isdf.pivot(index='sname', columns='cllclass', values='classof')

#count the number of times the country is listed
countclass = pivot2.count()
#convert series to dataframe
classdf = countclass.to_frame()

#make cll index to column
classdf.reset_index(inplace=True)

print(classdf.to_string())

                                                 cllclass  0
0                    Argentina|-38.416097|-63.616672|1912  1
1                        Armenia|40.069099|45.038189|1900  1
2                        Armenia|40.069099|45.038189|1917  1
3                        Armenia|40.069099|45.038189|1935  1
4                    Australia|-25.274398|133.775136|1905  1
5                    Australia|-25.274398|133.775136|1906  2
6                    Australia|-25.274398|133.775136|1907  2
7                    Australia|-25.274398|133.775136|1908  1
8                        Austria|47.516231|14.550072|1925  1
9                         Belgium|50.503887|4.469936|1888  1
10                       Brazil|-14.235004|-51.92528|1878  2
11                       Brazil|-14.235004|-51.92528|1879  2
12                       Brazil|-14.235004|-51.92528|1882  1
13                       Brazil|-14.235004|-51.92528|1885  1
14                       Brazil|-14.235004|-51.92528|1888  1
15                      

In [11]:
#rename column
classdf = classdf.rename(columns={0:'class_count'})
#split columns by delimeter
classdf[['cc','cclat','cclon','ccyear']] = classdf.cllclass.str.split("|",expand=True)

print(classdf)

                                    cllclass  class_count              cc  \
0       Argentina|-38.416097|-63.616672|1912            1       Argentina   
1           Armenia|40.069099|45.038189|1900            1         Armenia   
2           Armenia|40.069099|45.038189|1917            1         Armenia   
3           Armenia|40.069099|45.038189|1935            1         Armenia   
4       Australia|-25.274398|133.775136|1905            1       Australia   
..                                       ...          ...             ...   
149  United Kingdom|55.378051|-3.435973|1930            1  United Kingdom   
150  United Kingdom|55.378051|-3.435973|1932            1  United Kingdom   
151       Uruguay|-32.522779|-55.765835|1924            2         Uruguay   
152                         nan|nan|nan|1910            1             nan   
153                          nan|nan|nan|nan            1             nan   

          cclat       cclon ccyear  
0    -38.416097  -63.616672   1912  
1

In [12]:
#remove records where 'Class of' year is blank
classdf = classdf[classdf['ccyear'] != 'nan']
#change year to datetime so that it can be sorted in ascending order, this will default to YYYY-MM-DD, 
classdf['ccyear2'] = pd.to_datetime(classdf['ccyear'])
#change ccyear to just the YEAR without month and day
classdf['ccyear3'] = classdf['ccyear2'].dt.year
#sort records by year
classdf.sort_values(by=['ccyear3'], inplace = True)

print(classdf)

                                  cllclass  class_count           cc  \
99         Japan|36.204824|138.252924|1876            2        Japan   
22       Canada|56.130366|-106.346771|1877            1       Canada   
100        Japan|36.204824|138.252924|1877            1        Japan   
23       Canada|56.130366|-106.346771|1878            1       Canada   
10        Brazil|-14.235004|-51.92528|1878            2       Brazil   
..                                     ...          ...          ...   
66          China|35.86166|104.195397|1942            1        China   
117  Philippines|12.879721|121.774017|1943            1  Philippines   
146        Turkey|38.963745|35.243322|1943            1       Turkey   
90           India|20.593684|78.96288|1943            1        India   
47       Canada|56.130366|-106.346771|1943            1       Canada   

          cclat        cclon ccyear    ccyear2  ccyear3  
99    36.204824   138.252924   1876 1876-01-01     1876  
22    56.130366  -1

In [23]:
#animated map for student countries by each 'Class of' year
figclass = px.scatter_geo(
    classdf,
    lat=classdf['cclat'],
    lon=classdf['cclon'],
    color=classdf['ccyear'],
    size=classdf['class_count'],
    animation_frame = classdf['ccyear'],
    hover_name= classdf['cc'],
    hover_data= [classdf['class_count'], classdf['ccyear']],
    projection = 'natural earth',
    #projection = 'winkel tripel',
)

figclass.show()